# Sende Funktion

Natürlich, lassen wir uns die Funktion `send` Schritt für Schritt durchgehen. Ziel ist es, die genaue Funktion jeder Zeile zu verstehen.

---

### **Schritt-für-Schritt-Erklärung der `send`-Methode**

1. **Eingabeparameter und Constraints**
   ```python
   assert 0 < len(data) <= 252
   ```
   - **Erklärung**: Es wird überprüft, dass die Länge der zu sendenden Daten (`data`) zwischen 1 und 252 Bytes liegt. Dies liegt an der FIFO-Größe des RFM9x-Moduls, die auf maximal 256 Bytes beschränkt ist, wobei 4 Bytes für den Header reserviert sind.

2. **Wechsel in den "Idle"-Modus**
   ```python
   self.idle()
   ```
   - **Erklärung**: Bevor neue Daten gesendet werden, wird das Modul in den "Idle"-Modus versetzt. Dadurch wird der FIFO-Speicher geleert und sichergestellt, dass keine alten Daten im Speicher bleiben.

3. **FIFO-Startadresse setzen**
   ```python
   self._write_u8(_RH_RF95_REG_0D_FIFO_ADDR_PTR, 0x00)
   ```
   - **Erklärung**: Der interne FIFO-Speicher beginnt bei Adresse `0x00`. Diese Zeile stellt sicher, dass neue Daten von der Anfangsadresse des FIFO geschrieben werden.

4. **Header erstellen**
   ```python
   payload = bytearray(4)
   if destination is None:
       payload[0] = self.destination
   else:
       payload[0] = destination
   if node is None:
       payload[1] = self.node
   else:
       payload[1] = node
   if identifier is None:
       payload[2] = self.identifier
   else:
       payload[2] = identifier
   if flags is None:
       payload[3] = self.flags
   else:
       payload[3] = flags
   ```
   - **Erklärung**:
     - Der Header besteht aus 4 Bytes:
       1. `destination`: Zieladresse (Standard: Broadcast-Adresse, wenn nicht angegeben).
       2. `node`: Absenderadresse.
       3. `identifier`: Nachrichten-ID (z. B. Sequenznummer zur Verfolgung).
       4. `flags`: Zusätzliche Steuerinformationen.
     - Wenn keine Werte explizit angegeben werden, verwendet die Methode die Standardwerte der Instanz.

5. **Daten anhängen**
   ```python
   payload = payload + data
   ```
   - **Erklärung**: Der Header (4 Bytes) wird mit den eigentlichen Nutzdaten (`data`) kombiniert.

6. **Daten in den FIFO schreiben**
   ```python
   self._write_from(_RH_RF95_REG_00_FIFO, payload)
   ```
   - **Erklärung**: Der kombinierte Header und die Nutzdaten (`payload`) werden in den FIFO-Speicher des Moduls geschrieben. Dies ist der erste Schritt, um die Daten für die Übertragung vorzubereiten.

7. **Länge des Pakets setzen**
   ```python
   self._write_u8(_RH_RF95_REG_22_PAYLOAD_LENGTH, len(payload))
   ```
   - **Erklärung**: Das Modul muss wissen, wie viele Bytes es senden soll. Diese Zeile setzt die Länge des Pakets, einschließlich Header und Nutzdaten.

8. **Senden starten**
   ```python
   self.transmit()
   ```
   - **Erklärung**: Das Modul wird in den Sende-Modus (`TX_MODE`) versetzt. Der tatsächliche Übertragungsvorgang beginnt.

9. **Auf Abschluss der Übertragung warten**
   ```python
   start = time.ticks_ms()
   timed_out = False
   while not timed_out and not self.tx_done():
       if time.ticks_diff(time.ticks_ms(), start) >= self.xmit_timeout:
           timed_out = True
   ```
   - **Erklärung**:
     - Eine Schleife prüft, ob das Sende-Flag (`tx_done`) gesetzt wird, was anzeigt, dass die Übertragung abgeschlossen ist.
     - Die Schleife wird durch einen Timeout unterbrochen, falls die Übertragung zu lange dauert (`self.xmit_timeout`).

10. **Option: Zurück in den Empfangsmodus**
    ```python
    if keep_listening:
        self.listen()
    else:
        self.idle()
    ```
    - **Erklärung**:
      - Wenn `keep_listening=True` gesetzt ist, wechselt das Modul direkt in den Empfangsmodus (`RX_MODE`), um auf eingehende Nachrichten zu warten.
      - Andernfalls bleibt das Modul im "Idle"-Modus.

11. **Interrupts löschen**
    ```python
    self._write_u8(_RH_RF95_REG_12_IRQ_FLAGS, 0xFF)
    ```
    - **Erklärung**: Alle Interrupt-Flags werden gelöscht, um das Modul für zukünftige Vorgänge zurückzusetzen.

12. **Rückgabewert**
    ```python
    return not timed_out
    ```
    - **Erklärung**: Die Methode gibt `True` zurück, wenn die Übertragung erfolgreich abgeschlossen wurde. Bei einem Timeout wird `False` zurückgegeben.

---

### **Zusammenfassung des Ablaufs**
1. **Vorbereitung**:
   - Modul wird in den "Idle"-Modus versetzt.
   - Header und Nutzdaten werden kombiniert und in den FIFO-Speicher geschrieben.
2. **Sendevorgang**:
   - Das Modul wird in den Sende-Modus geschaltet.
   - Es wird auf den Abschluss der Übertragung gewartet.
3. **Nachbereitung**:
   - Optional: Wechsel in den Empfangsmodus.
   - Interrupts werden zurückgesetzt.

Falls du Details zu einem bestimmten Abschnitt brauchst, lass es mich wissen! 😊

# Senden mit Bestätigung

Die Funktion `send_with_ack` baut auf der `send`-Methode auf, fügt jedoch zusätzliche Logik hinzu, um sicherzustellen, dass die gesendeten Daten vom Empfänger bestätigt (ACK = Acknowledgment) werden. Sie ist für Szenarien gedacht, bei denen die zuverlässige Übertragung von Nachrichten wichtig ist. 

---

### **Schritt-für-Schritt-Erklärung der `send_with_ack`-Methode**

1. **Anzahl der Wiederholungen (Retries) festlegen**
   ```python
   if self.ack_retries:
       retries_remaining = self.ack_retries
   else:
       retries_remaining = 1
   ```
   - **Erklärung**: Die Methode verwendet die Eigenschaft `self.ack_retries`, um die maximale Anzahl von Sendeversuchen festzulegen.
   - Wenn `self.ack_retries` nicht definiert ist, wird nur ein Versuch durchgeführt.

2. **Initialisierung**
   ```python
   got_ack = False
   self.sequence_number = (self.sequence_number + 1) & 0xFF
   ```
   - **Erklärung**:
     - `got_ack` wird verwendet, um zu überprüfen, ob ein ACK vom Empfänger eingetroffen ist.
     - `self.sequence_number` wird inkrementiert und innerhalb des Bereichs 0–255 gehalten (8-Bit-Nummer). Diese Sequenznummer wird verwendet, um das ACK eindeutig einer Nachricht zuzuordnen.

3. **Wiederholungsschleife**
   ```python
   while not got_ack and retries_remaining:
   ```
   - **Erklärung**: Solange kein ACK empfangen wurde und noch Wiederholungsversuche verbleiben, wird die Nachricht erneut gesendet.

4. **Header mit aktueller Sequenznummer setzen**
   ```python
   self.identifier = self.sequence_number
   self.send(data, keep_listening=True)
   ```
   - **Erklärung**:
     - Die aktuelle Sequenznummer wird in den Header geschrieben.
     - Die `send`-Methode wird aufgerufen, um die Nachricht zu senden. Der Parameter `keep_listening=True` sorgt dafür, dass das Modul nach dem Senden direkt in den Empfangsmodus wechselt, um auf das ACK zu warten.

5. **ACK für Broadcast ignorieren**
   ```python
   if self.destination == _RH_BROADCAST_ADDRESS:
       got_ack = True
   ```
   - **Erklärung**: Wenn die Nachricht an die Broadcast-Adresse (`0xFF`) gesendet wurde, wird kein ACK erwartet. In diesem Fall wird die Schleife abgebrochen.

6. **Auf ACK warten**
   ```python
   ack_packet = self.receive(timeout=self.ack_wait, with_header=True)
   ```
   - **Erklärung**:
     - Die Methode `receive` wird aufgerufen, um nach eingehenden Paketen zu suchen.
     - Der Parameter `timeout=self.ack_wait` definiert die maximale Wartezeit auf ein ACK.
     - `with_header=True` sorgt dafür, dass auch der Header des empfangenen Pakets zurückgegeben wird.

7. **ACK überprüfen**
   ```python
   if ack_packet is not None:
       if ack_packet[3] & _RH_FLAGS_ACK:
           if ack_packet[2] == self.identifier:
               got_ack = True
               break
   ```
   - **Erklärung**:
     - Es wird überprüft, ob ein Paket empfangen wurde (`ack_packet is not None`).
     - `ack_packet[3] & _RH_FLAGS_ACK`: Prüft, ob das ACK-Flag im empfangenen Paket gesetzt ist.
     - `ack_packet[2] == self.identifier`: Stellt sicher, dass das ACK zur gesendeten Nachricht gehört (gleiche Sequenznummer).

8. **Warten vor erneutem Versuch**
   ```python
   if not got_ack:
       time.sleep_ms(self.ack_wait + int(self.ack_wait * random.random()))
   ```
   - **Erklärung**: Wenn kein ACK empfangen wurde, wird eine zufällige Verzögerung hinzugefügt, bevor die Nachricht erneut gesendet wird. Dies reduziert die Wahrscheinlichkeit von Kollisionen bei mehreren Geräten.

9. **Retries reduzieren und Retry-Flag setzen**
   ```python
   retries_remaining = retries_remaining - 1
   self.flags |= _RH_FLAGS_RETRY
   ```
   - **Erklärung**:
     - Die Anzahl verbleibender Wiederholungen wird um eins reduziert.
     - Das Retry-Flag im Header wird gesetzt, um anzuzeigen, dass die Nachricht erneut gesendet wird.

10. **Flags zurücksetzen**
    ```python
    self.flags = 0
    ```
    - **Erklärung**: Alle Header-Flags werden nach Abschluss des Prozesses zurückgesetzt.

11. **Ergebnis zurückgeben**
    ```python
    return got_ack
    ```
    - **Erklärung**: Die Methode gibt `True` zurück, wenn ein ACK empfangen wurde. Andernfalls wird `False` zurückgegeben.

---

### **Unterschiede zu `send`**
1. **ACK-Verarbeitung**:
   - `send_with_ack` wartet aktiv auf eine Empfangsbestätigung (ACK) und überprüft, ob das empfangene ACK zur gesendeten Nachricht gehört.
   - `send` sendet nur die Nachricht und gibt keinen Rückschluss auf den Empfang.

2. **Wiederholungen**:
   - `send_with_ack` versucht bei einem fehlenden ACK die Nachricht mehrmals zu senden, abhängig von der Konfiguration (`self.ack_retries`).

3. **Header-Flags**:
   - `send_with_ack` verwendet spezielle Header-Flags, um ACK und Retry-Informationen zu kommunizieren.

4. **Zuverlässigkeit**:
   - `send_with_ack` stellt sicher, dass die Nachricht den Empfänger erreicht (unter der Annahme, dass der Empfänger antworten kann).
   - `send` ist einfacher und schneller, garantiert aber nicht, dass die Nachricht erfolgreich empfangen wurde.

---

Wenn du weitere Details zu bestimmten Teilen brauchst oder etwas nicht klar ist, lass es mich wissen! 😊

# Recieve Funktion

Lass uns die `receive`-Methode Schritt für Schritt durchgehen. Diese Funktion ist dafür verantwortlich, eingehende Datenpakete zu empfangen und optional eine Empfangsbestätigung (ACK) zurückzusenden.

---

### **Schritt-für-Schritt-Erklärung der `receive`-Methode**

1. **Timeout-Wert setzen**
   ```python
   timed_out = False
   if timeout is None:
       timeout = self.receive_timeout
   ```
   - **Erklärung**: Die Methode verwendet entweder den angegebenen `timeout` oder den Standardwert (`self.receive_timeout`).
   - `timed_out` wird als Kontrollvariable für den Ablauf der Wartezeit initialisiert.

2. **Warten auf eingehende Daten**
   ```python
   if timeout is not None:
       self.listen()
       start = time.ticks_ms()
       while not timed_out and not self.rx_done():
           if time.ticks_diff(time.ticks_ms(), start) >= timeout:
               timed_out = True
   ```
   - **Erklärung**:
     - Das Modul wird in den Empfangsmodus (`RX_MODE`) versetzt.
     - Es wird aktiv überprüft, ob Daten empfangen wurden (`self.rx_done()`).
     - Die Schleife endet, wenn entweder Daten empfangen wurden oder der Timeout erreicht ist.

3. **Datenverarbeitung starten**
   ```python
   packet = None
   self.last_rssi = self.rssi
   self.idle()
   ```
   - **Erklärung**:
     - `packet` wird auf `None` gesetzt, um anzuzeigen, dass noch keine gültigen Daten empfangen wurden.
     - Die letzte RSSI (Signalstärke des empfangenen Pakets) wird gespeichert.
     - Das Modul wird in den "Idle"-Modus versetzt, um keine weiteren Pakete zu empfangen, während das aktuelle verarbeitet wird.

4. **CRC-Fehlerprüfung**
   ```python
   if not timed_out:
       if self.enable_crc and self.crc_error():
           self.crc_error_count += 1
       else:
   ```
   - **Erklärung**:
     - Wenn der Timeout nicht erreicht wurde, wird geprüft, ob die CRC-Prüfung aktiviert ist (`self.enable_crc`) und ob ein CRC-Fehler aufgetreten ist (`self.crc_error()`).
     - CRC-Fehler erhöhen den Zähler `self.crc_error_count`, und das Paket wird verworfen.

5. **Länge des empfangenen Pakets lesen**
   ```python
   fifo_length = self._read_u8(_RH_RF95_REG_13_RX_NB_BYTES)
   ```
   - **Erklärung**: Die Länge des empfangenen Pakets wird aus dem Register `_RH_RF95_REG_13_RX_NB_BYTES` gelesen.

6. **Paket aus dem FIFO-Speicher auslesen**
   ```python
   if fifo_length > 0:
       current_addr = self._read_u8(_RH_RF95_REG_10_FIFO_RX_CURRENT_ADDR)
       self._write_u8(_RH_RF95_REG_0D_FIFO_ADDR_PTR, current_addr)
       packet = bytearray(fifo_length)
       self._read_into(_RH_RF95_REG_00_FIFO, packet)
   ```
   - **Erklärung**:
     - Wenn die Paketlänge größer als 0 ist, wird die aktuelle FIFO-Adresse gelesen und als Startadresse gesetzt.
     - Die empfangenen Daten werden in das `packet`-Bytearray geladen.

7. **Interrupts löschen**
   ```python
   self._write_u8(_RH_RF95_REG_12_IRQ_FLAGS, 0xFF)
   ```
   - **Erklärung**: Alle Interrupt-Flags werden gelöscht, um das Modul für den nächsten Empfang vorzubereiten.

8. **Ungültige Pakete verwerfen**
   ```python
   if fifo_length < 5:
       packet = None
   else:
       if (
           self.node != _RH_BROADCAST_ADDRESS
           and packet[0] != _RH_BROADCAST_ADDRESS
           and packet[0] != self.node
       ):
           packet = None
   ```
   - **Erklärung**:
     - Pakete mit weniger als 5 Bytes werden verworfen, da sie keinen vollständigen Header enthalten.
     - Pakete, die nicht an die aktuelle Node-Adresse oder die Broadcast-Adresse gerichtet sind, werden ebenfalls verworfen.

9. **ACK senden (optional)**
   ```python
   elif (
       with_ack
       and ((packet[3] & _RH_FLAGS_ACK) == 0)
       and (packet[0] != _RH_BROADCAST_ADDRESS)
   ):
       if self.ack_delay is not None:
           time.sleep_ms(self.ack_delay)
       self.send(
           b"!",
           destination=packet[1],
           node=packet[0],
           identifier=packet[2],
           flags=(packet[3] | _RH_FLAGS_ACK),
       )
   ```
   - **Erklärung**:
     - Wenn `with_ack=True` gesetzt ist und das Paket kein ACK ist, wird eine Empfangsbestätigung (ACK) an den Absender zurückgeschickt.
     - Das ACK enthält die gleiche Sequenznummer wie das empfangene Paket.

10. **Header entfernen (optional)**
    ```python
    if not with_header and packet is not None:
        packet = packet[4:]
    ```
    - **Erklärung**: Wenn der Header nicht benötigt wird (`with_header=False`), wird er entfernt. Der Rest des Pakets (die Nutzdaten) wird zurückgegeben.

11. **Optional: Wieder in den Empfangsmodus wechseln**
    ```python
    if keep_listening:
        self.listen()
    else:
        self.idle()
    ```
    - **Erklärung**:
      - Wenn `keep_listening=True`, wechselt das Modul zurück in den Empfangsmodus.
      - Andernfalls bleibt es im "Idle"-Modus.

12. **Rückgabe des empfangenen Pakets**
    ```python
    return packet
    ```
    - **Erklärung**: Das empfangene Paket (mit oder ohne Header) wird zurückgegeben. Wenn kein Paket empfangen wurde, ist der Rückgabewert `None`.

---

### **Zusammenfassung der Funktion**
1. **Empfangsbereitschaft**: Das Modul wird in den Empfangsmodus versetzt, um auf eingehende Pakete zu warten.
2. **Paketprüfung**:
   - CRC-Fehler und ungültige Pakete werden verworfen.
   - Optional wird ein ACK zurückgesendet, um den Empfang zu bestätigen.
3. **Header-Verarbeitung**:
   - Der Header kann entfernt oder beibehalten werden, je nach Parameter `with_header`.
4. **Ergebnis**:
   - Gibt das empfangene Paket oder `None` zurück, wenn nichts empfangen wurde.

---

Wenn du noch spezifische Fragen zu einzelnen Schritten hast, lass es mich wissen! 😊